In [1]:
from dask.distributed import (
    Client, 
    progress as dask_progress,
    wait as dask_wait,
)
import os
import dask

# configure dashboard link to go over proxy
dask.config.set({"distributed.dashboard.link":
                 os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')+"proxy/{port}/status"});

In [2]:
from datacube.utils.rio import configure_s3_access

client = Client('tcp://dask-scheduler:8786')
display(client)

if False:
    client.restart()

if not client.get_metadata('odc_s3', False):
    # Configure GDAL for s3 access
    print('Configuring S3 read')
    configure_s3_access(aws_unsigned=True,  # works only when reading public resources
                        client=client)
    client.set_metadata('odc_s3', True)


Client Scheduler: tcp://dask-scheduler:8786 Dashboard: /proxy/8787/status,Cluster Workers: 1 Cores: 16 Memory: 128.85 GB


In [3]:
from botocore.credentials import ReadOnlyCredentials
import os

creds = ReadOnlyCredentials(
    os.environ['S3_WRITE_KEY'],
    os.environ['S3_WRITE_SECRET'], None)


In [4]:
from datacube.utils.cog import write_cog, to_cog
from datacube.utils.dask import save_blob_to_s3
from datacube import Datacube
import odc.algo
from odc.index import utm_region_code


dc = Datacube()

In [5]:
dss = dc.find_datasets(product='s2_gm_seasonal')

In [6]:
all_epsgs = set(ds.crs.epsg for ds in dss)
all_epsgs

{32749, 32750, 32751, 32752, 32753, 32754, 32755, 32756}

In [7]:
dss_by_epsg = {}
for ds in dss:
    dss_by_epsg.setdefault(ds.crs.epsg, []).append(ds)
list(dss_by_epsg)

[32749, 32750, 32751, 32752, 32753, 32754, 32755, 32756]

In [8]:
#client.restart()

In [9]:
from types import SimpleNamespace

def mk_task(epsg, 
            prefix='s3://dea-public-data-dev/temp/s2_gm/seasonal/v0/combined/thumbs/'):
    out_file = f'{prefix}s2_gm_2019_jja_{utm_region_code(epsg)}_rgba.tif'
    _dss = dss_by_epsg[epsg]
    
    xx = dc.load(datasets=_dss,
             measurements=['red', 'green', 'blue'],
             output_crs=_dss[0].crs,
             resolution=(-40,40),
             dask_chunks={'x': 5000, 'y': 5000}
            ).isel(time=0)

    rgba = odc.algo.to_rgba(xx, clamp=3000)
    cog_bytes = to_cog(rgba, 
                       blocksize=1024,
                       overview_levels=[2,4,8,16,32],
                       overview_resampling='bilinear')
    
    tif_save = save_blob_to_s3(cog_bytes,
                    out_file,
                    creds=creds,
                    ContentType="image/tiff")
    

    
    return SimpleNamespace(
        tif_save=tif_save,
        xx=xx,
        rgba=rgba,
        cog_bytes=cog_bytes,
        out_file=out_file,
    )

tasks = [mk_task(epsg) for epsg in dss_by_epsg]

In [10]:
import toolz

In [14]:
_keep = tt

In [16]:
for cch in toolz.partition_all(3, tasks[2:]):
    tt = []
    for task in cch:
        tt.append(client.compute(task.tif_save))

    for t in tt:
        print(t.result())


('s3://dea-public-data-dev/temp/s2_gm/seasonal/v0/combined/thumbs/s2_gm_2019_jja_51S_rgba.tif', True)
('s3://dea-public-data-dev/temp/s2_gm/seasonal/v0/combined/thumbs/s2_gm_2019_jja_52S_rgba.tif', True)
('s3://dea-public-data-dev/temp/s2_gm/seasonal/v0/combined/thumbs/s2_gm_2019_jja_53S_rgba.tif', True)
('s3://dea-public-data-dev/temp/s2_gm/seasonal/v0/combined/thumbs/s2_gm_2019_jja_54S_rgba.tif', True)
('s3://dea-public-data-dev/temp/s2_gm/seasonal/v0/combined/thumbs/s2_gm_2019_jja_55S_rgba.tif', True)
('s3://dea-public-data-dev/temp/s2_gm/seasonal/v0/combined/thumbs/s2_gm_2019_jja_56S_rgba.tif', True)


In [12]:
#client.restart()